# **Generalized Standard Material**

| Label | Task                        | Description                                                                                     | Category           | Developer       | Level |
|-------|-----------------------------|-------------------------------------------------------------------------------------------------|--------------------|-----------------|-------|
|       | Viscoelastic-Viscoplastic   | Serial coupling between visco-elasticity and visco-plasticity to analyze combined behavior.      | Understanding/Debugging | [MV]         |  A    |
|       | Kelvin Coupling             | Developing Kelvin-type coupling for enhanced theoretical and symbolic representation.           | Theory/Symbolics   | [RC]           |  B    |
|       | Prony Series Analysis       | Investigating Prony-series for time-dependent material behavior modeling.                       | Theory/Symbolics   | [RC]           |  B    |
|       | Legendre Transformations    | Exploring Legendre transforms between Helmholtz and Gibbs free energies for thermodynamic consistency. | Theory/Symbolics | [MV]           |  A    |
|       | GSM Expression Inspector    | Developing GSM Expression Inspector (GSM-EI) for debugging and inspecting symbolic expressions. | Tool Development   | [RC,VS]        |  A    |
| T-DV      | GSM Data Visualizer         | Creating GSM Data Visualizer (GSM-DV) for visualizing responses under basic load scenarios.      | Visualization      | [RC,VS]        |  A    |
|       | Model Workflow Optimization | Designing a workflow for model definition, including derivation, execution, and caching.         | Implementation     | [RC]           |       |
|       | Domain Multidimensionality  | Implementing multidimensional domain scanning for parameter sensitivity analysis.              | Implementation     | [RC]           |       |
|       | Thermal Effects Integration | Adding thermal terms to the model for temperature-dependent behavior analysis.                 | Implementation     | [RC]           | B     |
|       | Threshold definition with stress norm | Split the definition `f_expr` into `f_E_expr` - `f_R_expr`                 | Implementation     | [RC]           | B     |
|       | Split hardening and softening variables | As an alternative to `Sig_signs` use Ottossen structure                 | Implementation     | [RC]           | C     |


## T-DV

Define a base class - Loading Scenario and subclasses which introduce the basic scenarios 
 - constant loading, 
 - step loading, 
 - monotonic ascending, 
 - cyclic sinusoidal, 
 - cyclic saw teeth, 
 - cyclic with pauses 

# **Implementation-related remarks**

## Domain multidimensionality 

### Transition from sympy to numpy

**Symbolic Definition of Variables:** The inclusion of multiple control variables $\boldsymbol{\varepsilon}_a$ (`eps_a`) and internal variables $\boldsymbol{\mathcal{E}}_b$ (`Eps_b`) is achieved through a sequence of scalar, vector, and tensor definitions.

**Numerical Time Stepping:** This approach utilizes the multidimensional structure of `numpy` arrays. The generated `sympy` code accepts multidimensional arrays as input, evaluating along all extra dimensions beyond the symbolic function's native scope. Consequently, explicit loops over integration points ($i$) of finite element discretization elements ($E$) are not required.  


This folder contains a sequence of notebooks that drive the and document the development of the symbolic framework for the generalized standard material.

 Learned

  - Using reshape to handle the dimensional mismatch on sympy matrices in combination with lambdify.
  lambdify always returns a matrix even if the input dimensions of an array was a vector. This is particularly
  crucial for Sig_ 

  - Selection of inelastic indexes for return mapping brought a complication in the arrangement of the external 
    and internal state variables - the spatial fields are conveniently represented with the spatial index coming first and the state variable indexes last. On the other hand, lambdified functions require the state index coming first. Therefore, the method `get_state_n` uses `np.moveaxis` to put the last dimension of the state to the first position.
    Then, the inelastic subarrays idenfitied by the logical operator and `np.where`, i.e.
    `I = np.where(f > 0)`  need to be constructed as `bI = (slice(None), *I)`. This means for example as `Eps_k[bI]`.

  - Viscoplastic regularization calculates the final state via
    $$
    \mathcal{E}_{n+1} = \mathcal{E}_n + \gamma_{n+1} \left. \Phi \right|_{n}
    $$ 
    However, initial state the value of $\Phi$ is singular. Thus, a direct step from a zero state to an inelastic state results in division by zero. This might cause problems for high-modulus materials.

    2024-5-30 update: As a first attempt to make the viscoplastic regularization more robust for steps entering directly the inelastic regime - a two step version has been introduced defining the first step as subtraction of the identified amount of inelastic internal variables as
    $$
    \mathcal{E}_{k}^f = \mathcal{E}_k - \lambda_{k} \left. \Phi \right|_{k}
    $$
    and then taking this new fundamental state as a reference point and adding the internal variable increment according
    the flow magnitude corresponding to the retardation time   
    $$
    \mathcal{E}_{n} = \mathcal{E}_{k}^f + \gamma_{k} \left. \Phi \right|_{k}^f
    $$

Questions

  - what kinds of broadcasting in the calculation of response are possible? Can a parametric study be run in a vectorized form. For example, does it make sense that a simulation for a varied temperature is run within a single loop along the shared timeline?

  - How to handle the "as_explicit()" convention for internal variables? Should all internal calculations within gsm be done on an expanded representation of the state?

Todo

  - adapt the interface for the msx framework

# **Multi-dimensional implementation** 

2024-06-16:
In notebook [07_max_dissip_circular_domain_preliminary.ipynb](0700_max_dissip_circular_domain_preliminary.ipynb), there is an initial attempt to introduce the new material state identification based on the maximum dissipation principles. The gradient of the energy dissipation should be zero, the hessian matrix is derived but it contains zeros at the diagonal so that an inversion is not possible. Task: check if it can be used together with the scheme derived manually for the slide problem.  

# **Rate-dependency**

## Visco-elasticity versus visco-plasticity:

### Kelvin-Voigt Model and Viscoelasticity

- **Kelvin-Voigt Model**: This model is used to describe **viscoelastic** behavior, where:
  - The spring (elasticity) responds instantly to applied stress.
  - The dashpot (viscosity) introduces time-dependent strain.
  - Under constant stress, the strain increases as a function of time until it reaches an asymptote determined by the spring.

  The constitutive equation for the Kelvin-Voigt model is:
  $$
  \sigma = E \epsilon + \eta \dot{\epsilon}
  $$

- **Viscoelastic Behavior**: In this context:
  - At low loading rates, the elastic part dominates.
  - At high loading rates, the viscous part significantly influences the response.

### Visco-plasticity and Rheology

Visco-plasticity refers to materials that exhibit both viscous and plastic behavior, which involves yield stress beyond which permanent deformation occurs and time-dependent deformation characteristics.

**Note**: The Kelvin-Voigt model itself is not directly used to describe visco-plastic behavior because it does not incorporate a yield criterion. However, visco-plastic models often combine elements to incorporate both viscosity and plasticity.

#### Combining Viscoelastic and Plastic Components

To describe visco-plastic behavior, we often combine viscoelastic models with plasticity models:

1. **Generalized Maxwell Model or Kelvin-Voigt with a Plastic Slider**:
   - **Maxwell Model**: A spring and dashpot in series, often used with plastic components to capture visco-plasticity.
   - **Kelvin-Voigt Element with Plasticity**: Combining the viscoelastic Kelvin-Voigt component with a plastic slider element connected in series.

2. **Visco-plastic Constitutive Models**:
   - **Perzyna Model**: A well-known visco-plastic model where the plastic strain rate depends on a function of the stress exceeding the yield stress.
   - **Bingham Model**: Initially applied to fluids, this model incorporates a yield stress below which there is no flow, and above which the material behaves with both plastic and viscous characteristics.

**In summary**: The Kelvin-Voigt model is best suited for purely viscoelastic materials. To describe visco-plastic behavior, one needs to extend rheological models to include yield stress and plastic flow mechanisms, often through the combination of viscoelastic elements and yield/plasticity criteria.

## Viscoelastic Models and Reversibility

1. **Elastic Behavior (Spring)**:
   - **Character**: Reversible.
   - When the applied stress is removed, the material returns to its original shape.
   - Governed by Hooke’s Law: $ \sigma = E \epsilon $.

2. **Viscous Behavior (Dashpot)**:
   - **Character**: Reversible, but time-dependent.
   - When stress is applied, the dashpot allows the strain to continue accumulating over time, and when the stress is removed, the strain can relax back over time.
   - Governed by Newton’s Law of Viscosity: $ \sigma = \eta \dot{\epsilon} $.

### Kelvin-Voigt Model

- **Configuration**: Spring and dashpot in parallel.
- **Equation**: $\sigma = E \epsilon + \eta \dot{\epsilon}$.
- **Response**: The strain in the Kelvin-Voigt model is a combination of immediate elastic deformation and time-dependent viscous deformation.
- **Reversibility**: When the stress is removed, the elastic component immediately recovers its strain, and the viscous component gradually returns to the original state over time.

### Maxwell Model

- **Configuration**: Spring and dashpot in series.
- **Equation**: $\dot{\epsilon} = \frac{\sigma}{\eta} + \frac{\dot{\sigma}}{E}$.
- **Response**: The strain in the Maxwell model is a combination of both elastic deformation and time-dependent deformation (creep).
- **Reversibility**: The elastic component can recover its strain immediately when the stress is removed. However, the viscous component's deformation is more complex and can involve a time-dependent recovery (relaxation), but the overall behavior remains reversible.

## Key Characteristics of Viscoelasticity

- **Reversibility**: Deformation is fully recoverable upon the removal of stress.
- **Hysteresis**: The energy loss in a loading-unloading cycle is dissipated as heat, which is characteristic of the viscous component but still within a reversible framework.

## Contrast with Visco-plasticity

- **Permanent Deformation**: Visco-plastic materials exhibit plastic deformation (permanent) once the stress exceeds the yield stress.
- **Incorporation of Plasticity**: Models include a yield criterion (e.g., a slider) that causes irreversible deformation beyond the yield point.
- **Example**: A visco-plastic model often includes elements like springs, dashpots, and sliders in combination, where the slider introduces irreversibility by allowing permanent deformation after yielding.

## Implications

- **Viscoelastic Models**: Reversible deformation (elastic + time-dependent viscous behavior). No permanent change in shape after stress removal.
- **Visco-plastic Models**: Irreversible deformation (plastic deformation + time-dependent behavior). Permanent change in shape after yielding.

In conclusion, **viscoelastic models are fully reversible** as they do not incorporate mechanisms for permanent deformation, distinguishing them clearly from visco-plastic models, which do.

# **Entropy and temperature**

The relationship between entropy and temperature is fundamental to the laws of thermodynamics and statistical mechanics. Entropy, commonly denoted by \( S \), is a measure of the disorder or randomness in a system, while temperature, denoted by \( T \), is a measure of the average kinetic energy of the particles in a system.

### Thermodynamic Relationship

In classical thermodynamics, the relationship between entropy and temperature is described by:

$$ \frac{1}{T} = \left( \frac{\partial S}{\partial U} \right)_V $$

Here, $ U $ represents the internal energy of the system, and the partial derivative is taken at constant volume $ V $. This equation indicates that temperature is the thermodynamic conjugate variable associated with entropy.

### Fundamental Laws and Equations

#### 1. **First Law of Thermodynamics**:

$$
 dU = TdS - PdV 
 $$

where:
- $ dU $ is the change in internal energy.
- $ T $ is the temperature.
- $ dS $ is the change in entropy.
- $ P $ is pressure.
- $ dV $ is the change in volume.

In the context of reversible processes at constant volume, the relationship simplifies to:

$$ dU = TdS $$

This means the change in internal energy is directly proportional to the temperature and the change in entropy.

#### 2. **Second Law of Thermodynamics**:

The second law of thermodynamics states that the total entropy of an isolated system can never decrease over time. For a spontaneous process in an isolated system, the change in entropy is always positive or zero:

$$ \Delta S_{\text{total}} \geq 0 $$

This implies that entropy increases for irreversible processes and remains constant for reversible processes.

### Statistical Mechanics Perspective

From a statistical mechanics viewpoint, entropy can be related to the number of microscopic configurations (microstates) that correspond to a system's macroscopic state. According to Boltzmann's entropy formula:

$$ S = k_B \ln \Omega $$

where:
- $ S $ is entropy.
- $ k_B $  is Boltzmann's constant.
- $ \Omega $ is the number of accessible microstates.

### Temperature and Entropy in Various Processes

#### 1. **Isothermal Process** ($T$ constant):

For an isothermal process, $ T $ is constant, and the change in entropy can be given as:

$$ \Delta S = \frac{Q}{T} 
$$

where $ Q $ is the heat exchanged.

#### 2. **Adiabatic Process** ($Q = 0$):

For an adiabatic process, there is no heat exchange ($ Q = 0 $), and thus the entropy of the system remains constant:

$$ \Delta S = 0 $$

However, if an adiabatic process is irreversible, the entropy of the surroundings will increase, ensuring the total entropy still increases.

#### 3. **Reversible Process**:

For a reversible process, the change in entropy $ dS $ is related to the infinitesimal heat 
$ \delta Q_{\text{rev}} $ by:

$$ dS = \frac{\delta Q_{\text{rev}}}{T} $$

### Entropy and Temperature in Different Phases

- **Solid Phase**: At low temperatures, solids have low entropy because the particles are in well-defined positions.
- **Liquid Phase**: At moderate temperatures, liquids have higher entropy than solids because the particles are more disordered.
- **Gaseous Phase**: At high temperatures, gases have the highest entropy because the particles are in the most

# **Heat capacity**

In solids, the material parameter that relates entropy \(S\) and temperature \(T\) is often the specific heat capacity, particularly the heat capacity at constant volume, denoted as \(C_V\). The specific heat capacity is a measure of the amount of heat required to change the temperature of a unit mass of the material by one degree.

### Specific Heat Capacity at Constant Volume ($C_V$)

#### Definition:
- The heat capacity at constant volume, $C_V$, is defined as the amount of heat required to raise the temperature of a unit mass of a solid by one degree (keeping the volume constant).

$$ 
C_V = T \left( \frac{\partial S}{\partial T} \right)_V 
$$

#### Relationship with Entropy:
- The change in entropy $ dS $ can be directly related to the temperature change $ dT $ using the heat capacity at constant volume:

$$ dS = \frac{C_V}{T} dT $$

#### Integrating for Entropy Change:
- If the heat capacity $ C_V $ is constant over the temperature range of interest, the change in entropy from an initial temperature $ T_1 $ to a final temperature $ T_2 $ is given by:

$$
 \Delta S = \int_{T_1}^{T_2} \frac{C_V}{T} dT = C_V \ln \left( \frac{T_2}{T_1} \right) 
$$

### Understanding the Parameters:

1. **Specific Heat Capacity $ C_V $**:
   - **Units**: The units are typically J/(kg·K) or J/(mol·K) depending on whether specific heat per unit mass or molar heat capacity is used.
   - **Physical Meaning**: It indicates how much energy is required to change the temperature of the solid. For solids, it is often dominated by contributions from lattice vibrations (phonons).

2. **Entropy $ S $**:
   - **Units**: The units are J/K.
   - **Physical Meaning**: It represents the disorder or randomness in the system. For a solid, entropy increases with temperature because higher temperatures lead to more vibrational states being accessible to the atoms in the lattice.

### Low Temperature Behavior:

At low temperatures, the heat capacity of solids is described well by the Debye model, which accounts for the quantized nature of lattice vibrations (phonons). According to this model:

$$
C_V \propto T^3 \quad \text{(at low temperatures)} 
$$

### High Temperature Behavior:

At high temperatures, the heat capacity approaches a constant value predicted by the Dulong-Petit law for classical solids:

$$ 
C_V \approx 3R \quad \text{per mole} 
$$

where $ R $ is the universal gas constant.

### Practical Example:

For many simple solids, the heat capacity at constant volume $ C_V $ does not change drastically with temperature, allowing the entropy change to be approximated as:

$$ 
\Delta S \approx C_V \ln \left( \frac{T_2}{T_1} \right) 
$$

For more complex materials or specific temperature ranges, more detailed models and temperature-dependent specific heat capacity will be required.

### Summary:

In summary, the specific heat capacity at constant volume $ C_V $ is the key material parameter that connects entropy and temperature in solids. It describes how much heat is needed to change the temperature of the solid and, through integration, directly relates changes in temperature to changes in entropy.